In [51]:
#import packages
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import shape, Point
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import urllib.request, json
import requests
import urllib.parse
import datetime
import os
import dask.dataframe as dd
import calendar
from datetime import datetime, timedelta

from geopandas.tools import sjoin

import warnings
warnings.filterwarnings('ignore')

In [16]:
comp_dict_final = {'noise': ['noise - residential','noise survey', 'noise - commercial', 'noise - helicopter', 'collection truck noise', 'noise - vehicle', 'noise - house of worship', 'noise', 'noise - park', 'noise - street/sidewalk'],
                   'taxi complaints & inquiries': ['green taxi complaint', 'green taxi report', 'taxi compliment', 'dispatched taxi compliment', 'dispatched taxi complaint', 'for hire vehicle complaint', 'taxi complaint', 'taxi licensee complaint', 'taxi report', 'fhv licensee complaint', 'for hire vehicle report'],
                   'homelessness': ['homeless person assistance', 'homeless street condition', 'encampment', 'homeless encampment'],
                   'plaster/paint': ['paint/plaster','peeling paint', 'paint - plaster'],
                   'street/sidewalk/road condition': ['street condition', 'dep street condition', 'street light condition', 'root/sewer/sidewalk condition', 'dep sidewalk condition', 'dep bridge condition', 'curb condition', 'lot condition'],
                   'animal abuse': ['animal abuse', 'animal-abuse'],
                   'dead or damaged tree': ['dead tree', 'damaged tree', 'dead/dying tree'],
                   'waste & bulky item collection': ['institution disposal complaint', 'missed collection', 'missed collection (all materials)', 'residential disposal complaint', 'electronics waste appointment', 'seasonal collection', 'request large bulky item collection', 'request xmas tree collection', 'electronics waste'],
                   'litter basket complaints & inquiries': ['overflowing litter baskets', 'litter basket request', 'litter basket / request', 'litter basket complaint'],
                   'recycling basket complaints': ['recycling basket complaint', 'overflowing recycling baskets'],
                   'literature request': ['literature request', 'oem literature request', 'dor literature request', 'dof literature request', 'hpd literature request'], 
                   'hazardous materials': ['radioactive material', 'hazardous material', 'hazardous materials'],
                   'broken parking meter': ['broken parking meter', 'broken muni meter'],
                   'street sign complaints': ['highway sign - missing', 'highway sign - damaged', 'highway sign - dangling', 'street sign - dangling', 'street sign - missing', 'street sign - damaged'], 
                   'derelict vehicles': ['derelict vehicle', 'derelict vehicles'], 
                   'bus stop shelter complaint': ['bus stop shelter complaint', 'bus stop shelter placement'],
                   'illegal animal - sold/kept': ['illegal animal - sold/kept', 'illegal animal sold', 'illegal animal kept as pet'],
                   'home delivered meal complaint': ['home delivered meal - missed delivery', 'home delivered meal complaint'],
                   "building marshal's office": ["building marshal's office", 'building marshals office'],
                   'street sweeping complaint': ['sweeping/missed', 'sweeping/missed-inadequate', 'sweeping/inadequate'],
                   'fire alarm serive requests': ['fire alarm - new system', 'fire alarm - modification', 'fire alarm - reinspection', 'fire alarm - replacement'],
                   'general inquiry': ['general question', 'comments', 'general'],
                   'dof property complaints & inquiries': ['dof property - owner issue', 'dof property - update account', 'dof property - rpie issue', 'dof property - city rebate', 'dof property - reduction issue', 'dof property - payment issue', 'dof property - property value', 'dof property - request copy', 'dof property - state rebate'], 
                   'public advocate request': ['advocate - other', 'advocate-prop class incorrect', 'advocate-scrie/drie', 'advocate-prop refunds/credits', 'advocate-business tax', 'advocate-co-opcondo abatement', 'advocate-foreclosure', 'advocate - rpie', 'advocate - levy', 'advocate-commercial exemptions', 'advocate-personal exemptions', 'advocate-ubt', 'advocate - lien', 'advocate-property value'],
                   'dof parking requests': ['dof parking - payment issue', 'dof parking - request status', 'dof parking - request copy', 'dof parking - dmv clearance', 'dof parking - address update', 'dof parking - tax exemption'],
                   'public assembly': ['public assembly - temporary', 'public assembly'], 
                   'dhs advantage program inquiries': ['dhs advantage - tenant', 'dhs advantage - third party', 'dhs advantage -landlord/broker'],
                   'construction': ['general construction', 'construction'],
                   'electrical': ['electric', 'electrical'],
                   'unsanitary condition': ['unsanitary condition'],
                   'asbestos': ['asbestos', 'asbestos/garbage nuisance'],
                   'dirty conditions': ['dirty condition', 'dirty conditions'],
                  'snow' : ['snow','snw','snow or ice','snow removal'],
                  'ferry complaints & inquiries':['ferry inquiry','ferry permit','ferry complaint'],
                  'building facade complaints & inquiries':['facade insp safety pgm','facades']}

In [17]:
all311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
uniqueid = 'unique_key'
cat_clnm = 'agency_name'
subcat_clnm = 'complaint_type'
subsubcat_clnm = 'descriptor'
datetime_clnm = 'created_date'

In [1]:
def get_complaints_by_month(date_str, nta_nyc):
    year_month = datetime.strptime(date_str, '%Y-%m')
    next_month = year_month.replace(day=28) + timedelta(days=4)  # adds 4 days to get to the next month
    last_day = (next_month - timedelta(days=next_month.day)).strftime('%Y-%m-%d')
    url = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
    query = (url +'?'
            "$select= unique_key,complaint_type,latitude,longitude,created_date"
            "&$where=created_date between '"+date_str+'-01T00:00:00.000'+"' and '"+last_day+'T23:59:59.999'+"'"
            "&$limit=1000000")
    query = query.replace(" ", "%20")
    # Retry the query up to 5 times if it fails
    for i in range(5):
        try:
            response = urllib.request.urlopen(query)
            data = json.loads(response.read())
            
            df = pd.DataFrame(data)[['complaint_type', 'latitude', 'longitude', 'created_date', 'unique_key']]
            df = df[df['latitude'].notnull()]
            df = df[df['longitude'].notnull()]    
            # Step 2: Assign complaints to neighborhoods
            geometry = [Point(xy) for xy in zip(df.longitude.astype(float), df.latitude.astype(float))]
            crs = {'init': 'epsg:4326'}
            complaints = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
            complaints = gpd.sjoin(complaints, nta_nyc, how='inner',predicate = 'within')

            # Step 3: Clean up complaint types
            complaints['complaint_type_final'] = complaints['complaint_type'].str.lower()

            for complaint_type in complaints['complaint_type_final'].unique():
                stripped_complaint_type = complaint_type.strip().lower()

                if stripped_complaint_type in [val_item.strip().lower() for val in comp_dict_final.values() for val_item in val]:
                    key = [k for k, v in comp_dict_final.items() if stripped_complaint_type in [val_item.strip().lower() for val_item in v]][0]
                    complaints.loc[complaints['complaint_type_final'] == complaint_type, 'complaint_type_final'] = key

            # Step 4: Aggregate complaints by neighborhood and complaint type
            complaints = complaints.groupby(['ntacode', 'complaint_type_final']).agg({'unique_key': 'nunique'})
            complaints = complaints.unstack(level='complaint_type_final')
            complaints.columns = complaints.columns.droplevel()
            complaints.index.name = 'ntacode'
            complaints.reset_index(inplace=True)
            complaints['date'] = pd.to_datetime(date_str)
            complaints.set_index(['ntacode', 'date'], inplace=True)
        
        except Exception as e:
            print(f"Error fetching complaints for {date_str}: {e}")
            if i < 4:
                print(f"Retrying in {5 * (i+1)} seconds...")
                #time.sleep(5 * (i+1))
            elif i ==4: 
                print(f"Unable to load data for {date_str} even after 5 attempts. Returning NaN values for all NTACodes.")
                index = pd.MultiIndex.from_product([nta_nyc.ntacode.unique().tolist(), comp_dict_final.keys()], names=['ntacode', 'complaint_type_final'])
                df = pd.DataFrame({'count': np.nan}, index=index)
                complaints = df.unstack(level='complaint_type_final')
                complaints.columns = complaints.columns.droplevel()
                complaints.index.name = 'ntacode'
                complaints.reset_index(inplace=True)
                complaints['date'] = pd.to_datetime(date_str)
                complaints.set_index(['ntacode', 'date'], inplace=True)
    complaints = complaints.fillna(0)
    return complaints

In [20]:
#import nta shapefile
nta_url = 'https://data.cityofnewyork.us/resource/q2z5-ai38.json'

query = (nta_url+'?'
        "$select=*"
        "&$limit=10000")
query = query.replace(" ", "%20")
response = urllib.request.urlopen(query)
data = json.loads(response.read())


for d in data:
    d['geometry'] = shape(d['the_geom'])

#store in a geodataframe
nta_nyc = gpd.GeoDataFrame(data, geometry = 'geometry', crs = 'EPSG: 4326')
nta_nyc = nta_nyc.drop(columns = ['the_geom'])
nta_nyc.head()

,boro_code,boro_name,county_fips,ntacode,ntaname,shape_leng,shape_area,geometry
0,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ..."
1,2,Bronx,005,BX28,Van Cortlandt Village,21945.719299,25666124.5948,"MULTIPOLYGON (((-73.88705 40.88435, -73.88705 ..."
2,4,Queens,081,QN55,South Ozone Park,36708.1693055,82461393.7368,"MULTIPOLYGON (((-73.80577 40.68293, -73.80552 ..."
3,3,Brooklyn,047,BK40,Windsor Terrace,19033.6720656,14041667.99,"MULTIPOLYGON (((-73.98017 40.66115, -73.98021 ..."
4,3,Brooklyn,047,BK50,Canarsie,43703.6096661,82089678.6389,"MULTIPOLYGON (((-73.88834 40.64671, -73.88835 ..."


In [85]:
# Step 5: Loop over all months and combine results
start_date = pd.to_datetime('2010-01-01')
end_date = pd.to_datetime('2022-12-01')
complaints = pd.DataFrame()
for date in pd.date_range(start=start_date, end=end_date, freq='MS'):
    date_str = date.strftime('%Y-%m')
    print(date_str)
    monthly_complaints = get_complaints_by_month(date_str, nta_nyc)
    complaints = pd.concat([complaints, monthly_complaints])

2010-01
2010-02
2010-03
2010-04
2010-05
2010-06
2010-07
2010-08
2010-09
2010-10
2010-11
2010-12
2011-01
2011-02
2011-03
2011-04
2011-05
2011-06
2011-07
2011-08
2011-09
2011-10
2011-11
2011-12
2012-01
2012-02
2012-03
2012-04
2012-05
2012-06
2012-07
2012-08
2012-09
2012-10
2012-11
2012-12
2013-01
2013-02
2013-03
2013-04
2013-05
2013-06
2013-07
2013-08
2013-09
2013-10
2013-11
2013-12
2014-01
2014-02
2014-03
2014-04
2014-05
2014-06
2014-07
2014-08
2014-09
2014-10
2014-11
2014-12
2015-01
2015-02
2015-03
2015-04
2015-05
2015-06
2015-07
2015-08
2015-09
2015-10
2015-11
2015-12
2016-01
2016-02
2016-03
2016-04
2016-05
2016-06
2016-07
2016-08
2016-09
2016-10
2016-11
2016-12
2017-01
2017-02
2017-03
2017-04
2017-05
2017-06
2017-07
2017-08
2017-09
2017-10
2017-11
2017-12
2018-01
2018-02
2018-03
2018-04
2018-05
2018-06
2018-07
2018-08
Error fetching complaints for 2018-08: HTTP Error 429: Too Many Requests
Retrying in 15 seconds...


NameError: name 'time' is not defined

In [86]:
complaints.head()

,complaint_type_final,air quality,animal facility - no permit,animal in a park,appliance,asbestos,beach/pool/sauna complaint,best/site safety,bike/roller/skate chronic,blocked driveway,boilers,...,injured wildlife,fire alarm serive requests,hazmat storage/use,mobile food vendor,open flame permit,public assembly,rangehood,dept of investigations,gas station discharge lines,sprinkler - mechanical
ntacode,date,,,,,,,,,,,,,,,,,,,,,
BK09,2010-01-01,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BK17,2010-01-01,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,38.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BK19,2010-01-01,2.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,22.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BK21,2010-01-01,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,8.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BK23,2010-01-01,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
complaints.tail()

,complaint_type_final,air quality,animal facility - no permit,animal in a park,appliance,asbestos,beach/pool/sauna complaint,best/site safety,bike/roller/skate chronic,blocked driveway,boilers,...,injured wildlife,fire alarm serive requests,hazmat storage/use,mobile food vendor,open flame permit,public assembly,rangehood,dept of investigations,gas station discharge lines,sprinkler - mechanical
ntacode,date,,,,,,,,,,,,,,,,,,,,,
SI37,2018-07-01,1.0,0.0,0.0,6.0,0.0,0.0,2.0,0.0,50.0,0.0,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
SI45,2018-07-01,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,12.0,0.0,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
SI48,2018-07-01,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
SI54,2018-07-01,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,18.0,0.0,...,NaN,0.0,NaN,1.0,NaN,0.0,0.0,0.0,0.0,0.0
SI99,2018-07-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [88]:
complaints = complaints.fillna(0)
complaints.head()

,complaint_type_final,air quality,animal facility - no permit,animal in a park,appliance,asbestos,beach/pool/sauna complaint,best/site safety,bike/roller/skate chronic,blocked driveway,boilers,...,injured wildlife,fire alarm serive requests,hazmat storage/use,mobile food vendor,open flame permit,public assembly,rangehood,dept of investigations,gas station discharge lines,sprinkler - mechanical
ntacode,date,,,,,,,,,,,,,,,,,,,,,
BK09,2010-01-01,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BK17,2010-01-01,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,38.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BK19,2010-01-01,2.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,22.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BK21,2010-01-01,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BK23,2010-01-01,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
# Step 5: Loop over all months and combine results
start_date = pd.to_datetime('2018-08-01')
end_date = pd.to_datetime('2022-12-01')
complaints = pd.DataFrame()
for date in pd.date_range(start=start_date, end=end_date, freq='MS'):
    date_str = date.strftime('%Y-%m')
    print(date_str)
    monthly_complaints = get_complaints_by_month(date_str, nta_nyc)
    complaints = pd.concat([complaints, monthly_complaints])

2018-08
2018-09
2018-10
2018-11
2018-12
2019-01
2019-02
2019-03
2019-04
2019-05
2019-06
2019-07
2019-08
2019-09
2019-10
2019-11
2019-12
2020-01
2020-02
2020-03
2020-04
2020-05
2020-06
2020-07
2020-08
2020-09
2020-10
2020-11
2020-12
2021-01
2021-02
2021-03
2021-04
2021-05
2021-06
2021-07
2021-08
2021-09
2021-10
2021-11
2021-12
2022-01
2022-02
2022-03
2022-04
2022-05
2022-06
2022-07
2022-08
2022-09
2022-10
2022-11
2022-12


In [91]:
complaints = complaints.fillna(0)
complaints.tail()

,complaint_type_final,adopt-a-basket,air quality,animal abuse,animal facility - no permit,animal in a park,appliance,asbestos,beach/pool/sauna complaint,best/site safety,bike rack condition,...,obstruction,sanitation worker or vehicle complaint,storm,incorrect data,retailer complaint,transfer station complaint,face covering violation,private school vaccine mandate non-compliance,oil or gas spill,building drinking water tank
ntacode,date,,,,,,,,,,,,,,,,,,,,,
SI37,2022-12-01,0.0,4.0,3.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,...,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SI45,2022-12-01,1.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SI48,2022-12-01,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SI54,2022-12-01,0.0,2.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SI99,2022-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
complaints.to_csv('311_monthlycomplaints_nta.csv')

In [93]:
len(complaints)

10329